### V19: RCC (team + rafa) + SE + CENSO (team v2)

In [1]:
import pandas as pd
import re
import gc

In [2]:
def keep_matching_cols(df1, df2):
    keep_cols = list(set(df1.columns).intersection(set(df2.columns)))
    df1 = df1[keep_cols]
    df2 = df2[keep_cols]
    print(len(set(df1.columns) - set(df2.columns)) , len(set(df2.columns) - set(df1.columns)))
    return df1, df2

### Pre-Procesamiento RCC

In [3]:
rcc_train = pd.read_csv("/kaggle/input/interbank20/rcc_train.csv")
rcc_test = pd.read_csv("/kaggle/input/interbank20/rcc_test.csv")

In [4]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion_bins"] = pd.cut(rcc_train.condicion, bins)
rcc_test["condicion_bins"] = pd.cut(rcc_test.condicion, bins)

In [5]:
def makeCt(df, c, aggfunc=sum):
    try:
        ct = pd.crosstab(df.key_value, df[c].fillna("N/A"), values=df.saldo, aggfunc=aggfunc)
    except:
        ct = pd.crosstab(df.key_value, df[c], values=df.saldo, aggfunc=aggfunc)
    ct.columns = [f"{c}_{aggfunc.__name__}_{v}" for v in ct.columns]
    return ct

In [6]:
train = []
test = []
aggfuncs = [len, sum, min, max]
for c in rcc_train.drop(["codmes", "key_value", "saldo", "condicion"], axis=1):
    print("haciendo", c)
    train.extend([makeCt(rcc_train, c, aggfunc) for aggfunc in aggfuncs])
    test.extend([makeCt(rcc_test, c, aggfunc) for aggfunc in aggfuncs])

haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion_bins


In [7]:
train = pd.concat(train, axis=1)
test = pd.concat(test, axis=1)

In [8]:
gc.collect()

40

In [9]:
train, test = keep_matching_cols(train, test)

0 0


In [10]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

In [11]:
def pre_rcc(df):
    agg_key_mes = {
        'condicion_max'     : ('condicion', 'max'),
        'condicion_menos30' : ('condicion', lambda x: int(x.max() <= 30)),
        'condicion_menos60' : ('condicion', lambda x: int(x.max() <= 60 & x.max() > 30)),
        'condicion_menos90' : ('condicion', lambda x: int(x.max() <= 90 & x.max() > 60)),
        'condicion_mas90'   : ('condicion', lambda x: int(x.max() > 90)),
        'ifinan_count'      : ('cod_instit_financiera', 'count'),
        'rdirecto_max'      : ('RIESGO_DIRECTO', 'max'),
        'cdeudor_max'       : ('COD_CLASIFICACION_DEUDOR', 'max'),
        'saldo_sum'         : ('saldo', lambda x: x.abs().sum()),
    }

    agg_key = {
        'condicion_max_count'     : ('condicion_max', 'cumsum'),
        'condicion_menos30_count' : ('condicion_menos30', 'cumsum'),
        'condicion_menos60_count' : ('condicion_menos60', 'cumsum'),
        'condicion_menos90_count' : ('condicion_menos90', 'cumsum'),
        'condicion_mas90_count'   : ('condicion_mas90', 'cumsum'), 
        'ifinan_count_sum'        : ('ifinan_count_diff', 'cumsum'), 
    }
    
    df.drop(columns=['tipo_credito', 'PRODUCTO'], inplace=True)
    df['codmes'].replace(sorted(df.codmes.unique()), list(range(1,13)), inplace=True)

    df = df.groupby(['key_value', 'codmes']).agg(**agg_key_mes).reset_index()

    df['riesgo_diff']        = df.rdirecto_max ** 2 - df.cdeudor_max ** 2
    df['condicion_max_diff'] = df.groupby(['key_value']).condicion_max.diff().fillna(df['condicion_max'])
    df['ifinan_count_diff']  = df.groupby(['key_value']).ifinan_count.diff().abs().fillna(df['ifinan_count'])

    df2 = df.groupby(['key_value']).agg(**agg_key)

    df = pd.concat([df, df2], axis = 1).pivot(index = 'key_value', columns = 'codmes').reset_index().set_index('key_value')
    
    df.columns = [f'{f}_{s}' if s != '' else f'{f}'  for f, s in df.columns]    
    
    return df

In [12]:
train = pre_rcc(rcc_train).join(train)
test = pre_rcc(rcc_test).join(test)

In [13]:
del rcc_train, rcc_test
gc.collect()

20

### Pre-Procesamiento SUNAT

In [14]:
sunat_train = pd.read_csv("/kaggle/input/interbank20/sunat_train.csv")
sunat_test= pd.read_csv("/kaggle/input/interbank20/sunat_test.csv")

In [15]:
def pre_sunat(df):
    agg_key = {
        'tipcontribuyente_count'    : ('tipcontribuyente', 'nunique'),
        'tippersona_count'          : ('tippersona', 'nunique'),
        'ciiu_count'                : ('ciiu', 'nunique'),
        'ubigeo_count'              : ('ubigeo', 'nunique'),
        'condiciondomicilio_count'  : ('condiciondomicilio', 'nunique'),       
        'estadocontribuyente_count' : ('estadocontribuyente', 'nunique'),
        'codvia_count'              : ('codvia', 'nunique'),
        'codzona_count'             : ('codzona', 'nunique'),       
        'contabilidad_count'        : ('contabilidad', 'nunique'),  
        'facturacion_count'         : ('facturacion', 'nunique'),       
        'domiciliado_count'         : ('domiciliado', 'nunique'),  
        'comercioexterior_count'    : ('comercioexterior', 'nunique'),       
        'cargorele_count'           : ('cargorele', 'nunique'),          
        'codentidadtributo_count'   : ('codentidadtributo', 'nunique'),
        'fecalta_min'               : ('fecalta', 'min'),
        'actividad_vigente_flag'    : ('fecbaja', lambda x: int(x.isna().count() > 0)),        
        'actividad_finalizada_flag' : ('fecbaja', lambda x: int(x.count() > 0)),       
     }
    
    df1 = df.groupby(['key_value']).agg(**agg_key)
    df1['actividad_count'] = df1.iloc[:, 6:14].sum(axis=1)
    
#     df2 = pd.crosstab(df.key_value, df.estadotributo).add_prefix('estadotributo_')
#     df3 = pd.crosstab(df.key_value, df.ciiu).add_prefix('ciiu_')

#     df = df1.join(df2).join(df3)
    return df1

In [16]:
train = train.join(pre_sunat(sunat_train))
test = test.join(pre_sunat(sunat_test))

In [17]:
del sunat_train, sunat_test
gc.collect()

20

In [18]:
train, test = keep_matching_cols(train, test)

0 0


### Incluir SE & Censo

In [19]:
se_train = pd.read_csv("/kaggle/input/interbank20/se_train.csv", index_col="key_value")
se_test = pd.read_csv("/kaggle/input/interbank20/se_test.csv", index_col="key_value")
censo_train = pd.read_csv("/kaggle/input/interbank20/censo_train.csv", index_col="key_value")
censo_test = pd.read_csv("/kaggle/input/interbank20/censo_test.csv", index_col="key_value")

In [20]:
se_train.drop(columns=['cod_ubi'], inplace=True)
se_test.drop(columns=['cod_ubi'], inplace=True)

In [21]:
train = train.join(se_train).join(censo_train)
test = test.join(se_test).join(censo_test)

In [22]:
del se_train, se_test, censo_train, censo_test
gc.collect()

60

In [23]:
print('Train Columns:', len(set(train.columns)), '- Test Columns:', len(set(test.columns)))

Train Columns: 618 - Test Columns: 618


### Genero Output

In [24]:
train.to_csv('train-v20.gz', compression = 'gzip')
test.to_csv('test-v20.gz', compression = 'gzip')